# Initialize the framework

Import torch libraries and try to use the GPU device (if available)

In [1]:
import torch
from torch import nn

import random

# Try to use GPU device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using %s device" %(device))

Using cpu device


Mount Google Drive to load
* the lyrics dataset,
* the word2vec pretrained embedding dictionaries,
* the one hot encoding dictionary for the genres,
* the lyrics generator neural network



In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Load the dictionaries to convert words to indices and viceversa

In [3]:
#import pickle
import json

FILENAME_W2I = '/content/drive/MyDrive/DM project - NLP lyrics generation/Dictionaries/words2indices'
FILENAME_I2W = '/content/drive/MyDrive/DM project - NLP lyrics generation/Dictionaries/indices2words'

# Load a dictionary from a stored file converting keys to integers if needed
def load_dictionary(filename, convert_keys=False):
  #with open(filename + ".pkl", "rb") as f:
  #  return pickle.load(f)    

  with open(filename + ".json", "r") as f:
    d = json.load(f)

  if convert_keys:
    dd = {}
    for key, value in d.items():
      dd[int(key)] = value

    return dd
  
  return d

words2indices = load_dictionary(FILENAME_W2I)
indices2words = load_dictionary(FILENAME_I2W, convert_keys=True) # JSON stores keys as strings while here we expect integers

Load the word vectors tensor (word2vec embedding)

In [4]:
FILENAME = '/content/drive/MyDrive/DM project - NLP lyrics generation/Dictionaries/word_vectors.pt'

word_vectors = torch.load(FILENAME, map_location=device)

Load the one hot encoding dictionary for the genres

In [5]:
FILENAME = '/content/drive/MyDrive/DM project - NLP lyrics generation/Dictionaries/one_hot_encoding_genres'

one_hot_encoding_genres = load_dictionary(FILENAME)
NUMBER_GENRES = len(one_hot_encoding_genres)

Define vocabulary functions

In [6]:
# Get word from index
def get_word_from_index(idx):
  # Use get to automatically return None if the index is not present in the dictionary
  return indices2words.get(idx)

# Get index from word
def get_index_from_word(word):
  # Use get to automatically return None if the word is not present in the dictionary
  return words2indices.get(word)

# Get word vector from word
def get_word_vector(word):
  idx = get_index_from_word(word)
  return word_vectors[idx] if idx != None else None

Define the generator neural network

In [17]:
class Generator(nn.Module):

  def __init__(
      self,
      word_vectors: torch.Tensor,
      lstm_hidden_size: int,
      dense_size: int,
      vocab_size: int
  ):
    super().__init__()
    
    # Embedding layer
    self.embedding = torch.nn.Embedding.from_pretrained(word_vectors)
    
    # Recurrent layer (LSTM)
    self.rnn = torch.nn.LSTM(input_size=word_vectors.size(1), hidden_size=lstm_hidden_size, num_layers=1, batch_first=True)

    # Dense layer
    self.lin1 = torch.nn.Linear(dense_size, vocab_size) # Legacy
    #self.dense = torch.nn.Linear(dense_size, vocab_size)
    #torch.nn.init.uniform_(self.dense.weight)

    # Dropout function
    self.dropout = nn.Dropout(p=0.1)
		
		# Loss function
    self.loss = torch.nn.CrossEntropyLoss()
    
    self.global_epoch = 0
    
  def forward(self, x, y=None, states=None):
    # Split input in lyrics and genre
    lyrics = x[0]
    genres = x[1]

    # Embedding words from indices
    out = self.embedding(lyrics)

    # Recurrent layer
    out, states  = self.rnn(out, states)

    # Duplicate the genre vector associated to a sequence for each word in the sequence
    seq_length = lyrics.size()[1]

    if seq_length > 1:
      genres_duplicated = []
      for tensor in genres:
        duplicated = [list(tensor) for i in range(seq_length)]
        genres_duplicated.append(duplicated)

      genres = torch.tensor(genres_duplicated, device=device)
    else:
      # Just increment the genres vector dimension
      genres = genres.unsqueeze(0)


    # Concatenate the LSTM output with the encoding of genres
    out = torch.cat((out, genres), dim=-1)

    # Dense layer
    out = self.lin1(out) # Legacy
    #out = self.dense(out)

    # Use the last prediction
    logits = out[:, -1, :]
    
    # Scale logits in [0,1] to avoid negative logits
    logits = torch.softmax(logits, dim=-1)

    # Max likelihood can return repeated sequences over and over.
    # Sample from the multinomial probability distribution of 'logits' (after softmax). 
    # Return the index of the sample (one for each row of the input matrix) 
    # that corresponds to the index in the vocabulary as logits are calculated on the whole vocabulary
    sampled_indices = torch.multinomial(logits, num_samples=1)
    
    result = {'logits': logits, 'pred': sampled_indices, 'states': states}
    
    if y is not None:
      result['loss']     = self.loss(logits, y)
      result['accuracy'] = self.accuracy(sampled_indices, y.unsqueeze(-1))
      
    return result

  def accuracy(self, pred, target):
    return torch.sum(pred == target) / pred.size()[0]

Load the generator model

In [18]:
PATH = '/content/drive/MyDrive/DM project - NLP lyrics generation/Models/generator_model.pt'
#PATH = '/content/drive/MyDrive/DM project - NLP lyrics generation/generator_model_GAN.pt'

gen = Generator(
    word_vectors,
    lstm_hidden_size=256,
    dense_size=256+NUMBER_GENRES,
    vocab_size=len(word_vectors))

checkpoint = torch.load(PATH, map_location=device)
gen.load_state_dict(checkpoint['model_state_dict'])

# Try to move the model on the GPU 
if torch.cuda.is_available():
  gen.cuda()

# User input

Sort genres

In [19]:
genres = [key for key in one_hot_encoding_genres]
genres.sort()

Display input form

In [20]:
from ipywidgets import Layout, Box, Label, Dropdown, Text

print("Enter a word and a genre to generate a lyrics\n")
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

word_widget   = Text()
genres_widget = Dropdown(options=genres)

form_items = [
    Box([Label(value='Word'), word_widget], layout=form_item_layout), 
    Box([Label(value='Genre'), genres_widget], layout=form_item_layout),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='22%'
))
form

Enter a word and a genre to generate a lyrics



Box(children=(Box(children=(Label(value='Word'), Text(value='')), layout=Layout(display='flex', flex_flow='row…

Get user input

In [106]:
word  = word_widget.value
genre = genres_widget.value

In [107]:
##@title # Insert a word and a genre to generate a lyrics
#word  = "" #@param {type:"string", required: true}
#genre = "Country" #@param ["Country", "Electronic", "Folk", "Hip-Hop", "Indie", "Jazz", "Metal", "Pop", "Rock", "R&B"]

Preprocess the user input

In [108]:
# Split entered words on whitespaces to support also sequences of words
input_words = word.strip().split()

if not input_words:
  raise ValueError("No word entered")

# Check if every input word is present in the vocabulary (or in lowercase form)
for word in input_words:
  if word not in words2indices and word.lower() not in words2indices:
    raise ValueError("The entered word is not valid")

Generate the lyrics

In [109]:
TEXT_LENGTH = 100                # Truncate the text when the goal text length has been generated (hard truncation)
LINES = random.randrange(10, 50) # Truncate the text when the goal lines number has been generated (soft truncation)

states = None
text = ""
prev_word = ""
lines           = 0
generated_words = 0

word2capitalize = ["I", "I'm", "I'd"]
punctuation_subset = { '.', ',', ';', ':', '!', '?', ')', ']', '}', '$', '/', '…', '...', '..' }


# Iterate input words
for i in range(len(input_words)):
  w = input_words[i]

  # Check if the word is not present in the vocabulary in the current form
  if w not in words2indices:
    # Use the lowercase version (as it must be present in one of the two forms)
    input_words[i] = w.lower()

  # Check if this is the first word
  if i == 0:
    # Capitalize the first letter of the word
    w = w[0].upper() + w[1:]
    text = w
  else:
    text += ' ' + w

  prev_word = w

# Copy user input words to allow generating multiple lyrics with the same input
input_words_ = input_words.copy()

# One hot encode the genre
input_genre = one_hot_encoding_genres[genre]
input_genre = torch.tensor(input_genre, device=device).unsqueeze(0)


def generate_next_word(input_words, states=None):
  # Convert words to indices
  indices = [get_index_from_word(w) for w in input_words]
  indices = torch.tensor(indices, device=device).unsqueeze(0)

  y = gen((indices, input_genre), states=states)

  next_word_index = y['pred'].item()
  #print("next_word_index:", next_word_index)

  return get_word_from_index(next_word_index), y['states']


#for i in range(TEXT_LENGTH):
while lines < LINES:
  # Generate next word
  next_word, states = generate_next_word(input_words_, states)
  
  input_words_ = input_words_[1:]
  input_words_.append(next_word)

  #print("next word:", next_word)

  # Check if next word must be capitalized in the output text
  for word in word2capitalize:
    if next_word == word.lower():
      # Replace the generated word with the capitalized version
      next_word = word
      break

  # Check if previous word is newline (i.e. the generated word belongs to a new line) or a dot
  if prev_word == '\n' or prev_word == '.':
    # Capitalize the first letter of the generated word
    next_word = next_word[0].upper() + next_word[1:]
  
  # Check if previous word is newline or a parenthesis or next word is newline or punctuation
  if prev_word == '\n' or prev_word == '(' or next_word == '\n' or next_word in punctuation_subset:
    if next_word == '\n':
      # Update generated lines
      lines += 1

      # Check if the number of lines has been achieved
      if lines == LINES:
        break

    # Add the generated word to the output text without prepending a space
    text += next_word

  else:
    # Add the generated word to the output text prepending a space
    text += ' ' + next_word

  prev_word = next_word
  generated_words += 1


print("Word:", input_words)
print("Genre:", genre)
print("\nlines:", LINES)
print("generated words:", generated_words)
print("\nLyrics:")
print(text)

Word: ['saturday']
Genre: Rock

lines: 27
generated words: 233

Lyrics:
Saturday night
In the midnight room for the street
She walked dressed to themselves
You should have took her away, in a call as she cried for me
I was given to: about your mom. Punisher's bad news
You living why a girl I told you before
Don't you think that's the world?
Black space, smells blue?
There was her name and way: to the father french, out of the albums
Bring your blood underneath your gut bare
If you move your house
Don't know me
No I don't really want I love you (just make you to catch me!)
She's in the zone
I don't need you, half souls, forgive me, or nothing

You could ever leave it
When all fired this ray right into town
I don't even dream, fuck, step in my limousine
I can swear your ways
And leave me the questions I will know you are ready
In the wonderful dream
I like my real strange skies would tell me to find that girl I laid to rest
On the telephone I just steal trouble
My whole loving life
Thoug